In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import glob
import re

from sklearn.linear_model import LinearRegression
import seaborn

import pyperclip

# import tkinter as tk
# import tkinter.messagebox as messagebox

In [2]:
pd.set_option('display.max_columns', 50)

### ここだけ手動で設定。変更がないか毎度確認すること

In [3]:
subject_name_list = ["kumakura","kim","souma","fujii","tubota","toki"]
phase_name_list = ["rest", "practice", "boredom", "flow", "ultra", "overload"]
target_columns = ["bpm", "ibi", "lf", "hf","lf/hf","lf_diff", "hf_diff","lfhf_diff","FC3","FC4","FCz","mean_3ch","concentration_FC3","concentration_FC4","concentration_FCz","concentration_mean"]
target_phases = ["boredom","flow","ultra"]

folder_name = "regression_results_kubios_concentration_diff_all_overload"
pyperclip.copy(f"{folder_name}")

# tk.Tk().withdraw()
# messagebox.showinfo(f'{folder_name}フォルダを作成してください')



### 分析対象のファイルのパスを正規表現で取得

In [4]:
pathes = glob.glob("/Users/miyakooti/repositories/arai_MATLAB_program/csv/?_*/HRV_and_PLI.csv")
pathes.sort()
pathes

['/Users/miyakooti/repositories/arai_MATLAB_program/csv/0_kumakura/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/1_kim/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/2_souma/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/3_fujii/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/4_tubota/HRV_and_PLI.csv',
 '/Users/miyakooti/repositories/arai_MATLAB_program/csv/5_toki/HRV_and_PLI.csv']

### using all data

In [5]:
from scipy.stats import pearsonr
import statistics

In [6]:
for i,target_phase in enumerate(target_phases):
    
    for j,path in enumerate(pathes):
        if "kim" in path:
            continue
        df = pd.read_csv(path,index_col=0).fillna(0)

        df = df.loc[[target_phase]] # seriesとして取り出したいときはこっち
        if j == 0 and i == 0:
            print("flow_datasetを初期化しました")
            flow_dataset = df
        else:
            flow_dataset = pd.concat([flow_dataset, df], axis=0)

flow_dataset

flow_datasetを初期化しました


,bpm,ibi,sdnn,sdsd,rmssd,pnn20,pnn50,hr_mad,sd1,sd2,s,sd1/sd2,breathingrate,vlf,lf,hf,lf/hf,p_total,vlf_perc,lf_perc,hf_perc,lf_nu,hf_nu,lf_diff,hf_diff,lfhf_diff,FpZ,FC3,FC4,FCz,O1,O2,Fp2,mean_all,mean_3ch,concentration_FC3,concentration_FC4,concentration_FCz,concentration_mean,difficulty,fast,fulfillment,absorb,questionnaire_average
boredom,75.368816,796.085213,41.400093,17.968812,33.557870,0.610553,0.130653,24,23.728891,53.486937,3987.264510,0.443639,0.346315,232.041729,318,315,1.000,856.905303,27.079040,33.931523,38.989437,46.531921,53.468079,-84.376808,148.471441,-0.442601,0.28649,0.43996,0.43761,0.54694,0.17318,0.15961,0.23876,0.326079,0.474837,0.272161,0.092863,0.222104,0.195709,2,2.0,5.0,2.0,2.75
boredom,63.403782,946.315790,53.660997,24.508470,47.306947,0.750733,0.346041,35,33.450970,68.034150,7149.662688,0.491679,0.247188,985.210924,934,1088,0.859,2869.012787,34.339719,25.007509,40.652772,38.086204,61.913796,-304.607575,188.070821,-0.310320,0.19084,0.36094,0.37935,0.43916,0.48950,0.44796,0.22319,0.361563,0.393150,0.051840,-0.010917,0.026382,0.022435,5,3.0,3.0,3.0,3.50
boredom,70.832500,847.068783,51.667645,18.167883,33.520254,0.602122,0.111406,28,23.701771,69.102351,5145.451513,0.342995,0.124926,1000.248023,1296,309,4.190,2660.157941,37.601077,52.544882,9.854041,84.207995,15.792005,-991.823110,110.881925,-4.244014,0.34038,0.47183,0.38638,0.64152,0.28792,0.39155,0.28821,0.401113,0.499910,0.267063,0.372362,0.157989,0.265805,3,1.0,5.0,5.0,3.50
boredom,88.815050,675.561181,40.001179,18.624539,30.105663,0.473573,0.101480,30,21.287512,52.278441,3496.209181,0.407195,0.124918,230.914240,604,229,2.639,1144.402893,20.177705,60.184245,19.638050,75.397788,24.602212,-132.790635,-8.266491,-0.496399,0.44885,0.60714,0.64297,0.73292,0.28617,0.39602,0.46605,0.511446,0.661010,-0.013874,0.047647,-0.007030,0.008914,3,2.0,4.0,2.0,2.75
boredom,71.749702,836.240409,46.771508,15.257885,24.677198,0.394872,0.041026,26,17.448147,63.660570,3489.552176,0.274081,0.122335,702.413113,963,202,4.770,2153.481543,32.617559,58.812408,8.570033,87.281504,12.718496,-351.866804,-49.131153,-0.108557,0.34840,0.55059,0.56533,0.61080,0.30801,0.45166,0.36296,0.456821,0.575573,-0.013874,0.047647,-0.007030,0.008914,1,1.0,2.0,1.0,1.25
flow,79.462990,755.068493,45.496886,17.074905,30.303453,0.552369,0.093516,30,21.427739,60.626869,4081.232643,0.353436,0.131944,744.117782,290,80,3.610,1043.384149,71.317720,21.756989,6.925292,75.855156,24.144844,-20.624755,410.316596,-2.713997,0.26200,0.42757,0.43541,0.37519,0.66146,0.52691,0.25446,0.420429,0.412723,0.290671,0.095183,0.407868,0.264574,7,7.0,7.0,7.0,7.00
flow,67.177316,893.158637,55.573726,20.581022,38.907855,0.674118,0.195294,32,27.511960,73.648121,6365.508048,0.373560,0.315760,1684.541005,375,461,0.815,2715.033129,62.044952,17.675763,20.279285,46.570256,53.429744,-67.041765,803.814751,-0.566788,0.38484,0.26153,0.31830,0.55653,0.23826,0.16282,0.33451,0.322399,0.378787,0.151255,0.050135,-0.090986,0.036801,5,6.0,6.0,6.0,5.75
flow,71.024566,844.778135,40.814629,15.735791,28.317559,0.515021,0.061159,24,20.023450,54.132461,3405.230837,0.369897,0.139564,565.669734,290,167,1.730,1240.273592,45.608464,35.932564,18.458972,66.062785,33.937215,-39.708354,144.073228,-0.858314,0.24192,0.44200,0.50466,0.64272,0.26723,0.35481,0.28993,0.391896,0.529793,0.296889,0.254080,0.156784,0.235918,7,6.0,6.0,6.0,6.25
flow,87.377865,686.672760,46.869230,18.434993,30.624127,0.487649,0.093321,33,21.654505,62.669735,4263.399226,0.345534,0.119805,617.281711,224,80,2.802,1161.093902,53.163806,36.938615,9.897579,78.867671,21.132329,127.067599,101.551737,-1.163810,0.41807,0.44068,0.47894,0.53998,0.39422,0.33732,0.40138,0.430084,0.486533,0.152582,0.211672,0.185907,0.183387,5,7.0,6.0,7.0,6.25
flow,72.643550,825.950820,41.466246,14.500607,24.770080,0.420868,0.035170,27,17.514995,55.952802,3078.801861,0.313032,0.127594,531.814471,446,191,2.340,1141.509628,46.588698,39.348650,14.062653,73.671017,26.328983,465.478921,-25.103605,3.955912,0.49536,0.66217,0.6586

In [7]:
# 保存用dict
export_data = {    
    "target": target_columns,
    "mean": [],
    "SD": [],
    "R": [],
    "p": [],
}

# それぞれの説明変数に対して，，，
for target_column in target_columns:
    print(f"{target_column}の相関係数を計算します．")

    x = flow_dataset[target_column]
    y = flow_dataset[["questionnaire_average"]]

    # ピアソンの積率相関係数を計算        
    R, p = pearsonr(x, y)

    export_data["mean"].append(statistics.mean(x))
    export_data["SD"].append(statistics.pstdev(x))
    export_data["R"].append(R[0])
    export_data["p"].append(p)
        
df = pd.DataFrame(export_data)
    
# 保存
save_path = f"/Users/miyakooti/repositories/arai_MATLAB_program/csv/{folder_name}/pearsonr.csv"
df.to_csv(save_path)  

bpmの相関係数を計算します．
ibiの相関係数を計算します．
lfの相関係数を計算します．
hfの相関係数を計算します．
lf/hfの相関係数を計算します．
lf_diffの相関係数を計算します．
hf_diffの相関係数を計算します．
lfhf_diffの相関係数を計算します．
FC3の相関係数を計算します．
FC4の相関係数を計算します．
FCzの相関係数を計算します．
mean_3chの相関係数を計算します．
concentration_FC3の相関係数を計算します．
concentration_FC4の相関係数を計算します．
concentration_FCzの相関係数を計算します．
concentration_meanの相関係数を計算します．


# 参考文献
- https://www.st-hakky-blog.com/entry/2018/01/30/004659